In [1]:
cd "/content/drive/MyDrive/Colab Notebooks/Federated_mnist"

/content/drive/MyDrive/Colab Notebooks/Federated_mnist


In [2]:
# install syft

!pip install syft==0.1.29a1

     |████████████████████████████████| 327kB 13.9MB/s 
     |████████████████████████████████| 2.6MB 27.8MB/s 
     |████████████████████████████████| 204kB 60.5MB/s 
     |████████████████████████████████| 2.7MB 48.3MB/s 
     |████████████████████████████████| 1.8MB 56.4MB/s 
     |████████████████████████████████| 512kB 56.2MB/s 
     |████████████████████████████████| 276kB 54.6MB/s 
     |████████████████████████████████| 110.5MB 31kB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 512kB 36.0MB/s 
     |████████████████████████████████| 3.8MB 55.4MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.8.1-cp36-cp36m-linux_x86_64.whl size=1206050 sha256=740e094a62a77fb6da92cce06ec221686aa96611d3ff21dc3191acbc5abf5552
  Stored in directory: /root/.cache/pip/wheels/b2/c3/7b/401864dafa21db92ecb4b1b73b0f6f66d0d9f0a7a1dce5e3a7
  Created 

Requirements: 

torch==1.1.0

torchsummary==1.5.1

torchtext==0.3.1

torchvision==0.3.0

syft==0.1.29a1

numpy==1.16.4

matplotlib==3.0.3

In [4]:
!pip install torch==1.1.0

     |████████████████████████████████| 676.9MB 21kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.1.0 which is incompatible.
ERROR: syft 0.3.0 has requirement torch>=1.5, but you'll have torch 1.1.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [3]:
!pip install torchsummary==1.5.1

In [4]:
!pip install torchtext==0.3.1

In [5]:
!pip install torchvision==0.3.0

In [9]:
!pip install pillow==6.1 
# Pillow 7.0.0 removed PILLOW_VERSION, you should use __version__ in your own code instead


     |████████████████████████████████| 2.1MB 12.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [2]:
import torch as th
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.5.so'


In [3]:

#Method to create 10 virtual workers and move to a list of workers
# add more workers based on the workers(number of workers which trains the model with private data) you require. 
def create_workers():
  workers = []
  cartman = sy.VirtualWorker(hook, id = "cartman")
  workers.append(cartman)
  kyle = sy.VirtualWorker(hook, id = "kyle")
  workers.append(kyle)
  kenny = sy.VirtualWorker(hook, id = "kenny")
  workers.append(kenny)
  stan = sy.VirtualWorker(hook, id = "stan")
  workers.append(stan)
  butters = sy.VirtualWorker(hook, id = "butters")
  workers.append(butters)
  wendy = sy.VirtualWorker(hook, id = "wendy")
  workers.append(wendy)
  heidi = sy.VirtualWorker(hook, id = "heidi")
  workers.append(heidi)
  bebe = sy.VirtualWorker(hook, id = "bebe")
  workers.append(bebe)
  nichole = sy.VirtualWorker(hook, id = "nichole")
  workers.append(nichole)
  patty = sy.VirtualWorker(hook, id = "patty")
  workers.append(patty)
  return workers

In [4]:
#Method to clear out every tensor stored in the list of virtual workers
def clear_workers(workers):
  for worker in workers:
    worker.clear_objects()

In [5]:
# train loader and test loader for data splitting
#Method to split the mnist test dataset into the various workers and also to load the mnist test dataset into a test loader
def create_federated_and_test_loaders(workers, trainset, testset):
  federated_train_loader = sy.FederatedDataLoader(
      trainset.federate(workers), 
      batch_size=32, shuffle=True)

  test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
  return federated_train_loader, test_loader

In [6]:
def create_train_federated_models(workers, loader, lr = 0.12, epoch = 5):
  #sends model to first virtual worker
  virtual_model = classifier().send(workers[0])
  optimizer = optim.SGD(virtual_model.parameters(), lr)
  criterion = nn.NLLLoss()
  for n in range(epoch):
    
    #Integer to keep up with first index.
    i = 0
    
    #Integer to keep up with current worker while training
    j = 0
    
    #Integer to count number of mini-batches per worker
    n_mbatch = 0
    
    #Variable to keep up with current worker while looping
    dbLoc = None
    
    #Variable to store cummulative loss.
    cum_loss = 0
    
    
    for batch_idx, (imgs, labels) in enumerate(loader):
      
      #condition to set dbLoc to the first worker
      if i == 0:
        i = 2
        dbLoc = imgs.location
        
      #condition to change dbLoc if img is stored on a different worker and also calculate loss
      if dbLoc is not imgs.location:
        print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss / n_mbatch, n+1))
        dbLoc = imgs.location
        j += 1
        
        #Moving the model to a new worker
        virtual_model.move(dbLoc)
        
        #Resetting the cummulative loss and batch count to zero for new worker
        cum_loss = 0
        n_mbatch = 0

      optimizer.zero_grad()
      output = virtual_model.forward(imgs)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      cum_loss +=  loss.get().item()
      n_mbatch += 1
    print("The total loss for {0} is {1}".format(workers[j].id, cum_loss / n_mbatch))
    
    #Moving the model to the first worker if training would occur again
    if (n < (epoch - 1)):
      virtual_model.move(workers[0])
  return virtual_model

In [7]:
#Method to return the model to the central machine
def create_central_model(model):
  return model.get()

In [8]:
#Method to analyze the private database with the trained model
def analyze_model(model, loader):
  print("Running on ", "cpu")
  cum_perc = 0
  for imgs, labels in loader:
    with th.no_grad():
      ps =  th.exp(model.forward(imgs))
    top_p, top_class = ps.topk(1, dim = 1)
    prob = top_class == labels.view(*top_class.shape)
    prob = prob.float()
    cum_perc += prob.mean().float()
  print("The accuracy of the model is {0}%".format((cum_perc / len(loader)) * 100))

In [9]:
#Classifier for creating the models
class classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.log_softmax(self.fc5(x), dim = 1)   
    return x

In [10]:
# Application of transforms to normalize the mnist data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

9920512it [00:01, 7656078.00it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 117617.27it/s]           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2423787.13it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 34064.70it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [11]:
workers = create_workers()
clear_workers(workers)
federated_loader, test_loader = create_federated_and_test_loaders(workers, mnist_trainset, mnist_testset)

In [12]:
virtual_model = create_train_federated_models(workers, federated_loader, epoch = 2)

The total loss for cartman for epoch 1 is 2.307324820376457
The total loss for kyle for epoch 1 is 2.3089127185496876
The total loss for kenny for epoch 1 is 2.312258960084712
The total loss for stan for epoch 1 is 2.3074064013805797
The total loss for butters for epoch 1 is 2.3090646266937256
The total loss for wendy for epoch 1 is 2.3092944457175886
The total loss for heidi for epoch 1 is 2.307045430579084
The total loss for bebe for epoch 1 is 2.310981546310668
The total loss for nichole for epoch 1 is 2.310140146854076
The total loss for patty is 2.3116971216303237
The total loss for cartman for epoch 2 is 2.3074453726727913
The total loss for kyle for epoch 2 is 2.3089565482545407
The total loss for kenny for epoch 2 is 2.3121859393221267
The total loss for stan for epoch 2 is 2.3074388225027858
The total loss for butters for epoch 2 is 2.3092057007424374
The total loss for wendy for epoch 2 is 2.3092764045329806
The total loss for heidi for epoch 2 is 2.3072677077131067
The total

In [13]:
central_model = create_central_model(virtual_model)

In [14]:
analyze_model(central_model, test_loader)

Running on  cpu
The accuracy of the model is 9.802946090698242%
